In [ ]:
# %matplotlib widget
import pandas as pd
import numpy as np
import pw_helpers
import matplotlib.pyplot as plt

# Introduction to pandas

The observations have been made by a GNSS receiver placed on the roof of a building at CNES (Toulouse), and recorded in `RINEX` format (Receiver Independent Exchange format). The RINEX file has been post-processed by `PRX` (processed RINEX, [github](https://github.com/jtec/prx)), a program developped by GNSS enthusiasts, to compute all the parameters required to compute a GNSS position. This produced a file, saved in `CSV` format (comma-separated values), thanks to the following commands:

```
poetry env use 3.12
poetry install
poetry shell
python src/prx/main.py --observation_file_path TLSE00FRA_R_20240010100_15M_30S_MO.rnx
```

We are going to import this file in a data structure used by the `Pandas` package: the `pandas.DataFrame`.

In [ ]:
# load data
data_prx = pw_helpers.prx_csv_to_pandas(
    "./data/TLSE00FRA_R_20240010100_01H_30S_MO.csv",
    # "./data/TLSE00FRA_R_20240010000_01D_30S_MO.csv",
)

# show the first 5 lines
data_prx.head()

The resulting `pd.DataFrame` contains a data table.  
The `pd.DataFrame` has an index (the first column), which is here just an integer. It could be any other type of data, such as a categorical value or a timestamp.  
Each rows contains a single GNSS observation and all the associated data. You can see the list of all the available parameters as column headers.

In [ ]:
data_prx.columns


We are now going to learn how to use `pandas` feature in order to
- access the data in the columns
- filter out some rows
- perform some operations
- plot the data

# Accessing data
To access a row, you can use the `.iloc[index_val]` method.  
> ***Instruction:*** Try to access:
> - the first line of `data_prx`
> - the first two lines of `data_prx`

Note: remember that Python uses a zero indexing convention.

In [ ]:
# first_line =...
# pw_helper.check_answer(first_line.L_obs_cycles,207663744.792)

# first_two_lines = ...
# pw_helpers.check_answer(first_two_lines.iloc[0:2].L_obs_cycles,[207663744.792,168743803.031])

To access a particular column, you can call it either
- with square brackets: `data_prx["C_obs_m"]`
- or with a dot: `data_prx.C_obs_m`

This will return another `pandas` structure, called the `pandas.Series`, sharing the same index.
A `pandas.Series` is just a `DataFrame` with a single column.

> ***Instruction:***
> - Using `.head()`, display the 5 first values of the `prn` column of `data_prx`.

In [ ]:
# prn_series = ...

You can also access several columns by putting a `list` of column names between brackets.
This will return another `DataFrame` with several columns.

> ***Instruction:*** 
> - Using `.head()`, display the 5 first values of the `prn`, `constellation` and `rnx_obs_identifier` columns of data_prx

Note: the RINEX observation identifiers can be found in section 5.1 (p22) of the document `resources/rinex_3.04.pdf`.

In [ ]:
# prn_constellation_obsid = ...

# Conditional filtering
`pandas` allow to quickly filter the rows thanks to the `.loc[condition]` method, where `condition` is a `Series` of boolean.

For example, you can keep only the observations with high C/No thanks to the following line:  
`data_high_snr = data_prx.loc[ data_prx.S_obs_dBHz > 45 ]`

> ***Instruction:***
> - Try this command and display the number of observations before and after filtering.

In [ ]:
# data_high_snr = 

# print(f"There are {len(data_prx)} observations in the original dataset")
# print(f"There are {len(data_high_snr)} observations with a C/No above 45 dbHz")

You can also check if one column is within a list of values with the function `Series.isin(list)`.

> ***Instruction:***  
> - Display the number of observations from the constellations GPS (`constellation=="G"`) and Galileo (`constellation=="E"`)

In [ ]:
# data_gps = ...
# data_gal = ...
# data_gps_gal = ...

# print(f"There are {len(data_gps)} GPS observations and {len(data_gal)} GAL observations.")
# print(f"This sums up to {len(data_gps_gal)} GPS and GAL observations.")

You can use several conditions when filtering with `.loc[(condition_1) & (condition_2)]`

> ***Instruction:***
> - Extract the observations containing only GPS and L1 C/A observations (corresponding `rnx_obs_identifier` is `"1C"`)

In [ ]:
# data_gps_c1c = ...

# print(f"There are {len(data_prx)} observations in the original dataset")
# print(f"There are {len(data_gps_c1c)} GPS L1 C/A observations")

If you look at the index of the resulting `DataFrame`, it is now a discontinuous integer series. It may be difficult to manipulate.  
You can use the method `.reset_index(drop=True)` to re-initialize the index.

> ***Instruction:***
> - Display the first 3 rows (with `.head(3)`) before and after re-initializing the index.

In [ ]:
# Display the first 3 rows before re-initializing the index

In [ ]:
# Re-initialize the index
# data_gps_c1c = ...

# Display the first 3 rows after re-initializing the index

# Operation on `DataFrame`
You can apply some usual operations on whole `Series`, such as `mean(), sum(), min(), max(), median(), idxmax(), idxmin(), quantile([q1,q2,…])`, etc.


> ***Instruction:***
> - Compute the minimum and maximum values of the code and doppler observation on `data_gps_c1c`.

> ***Hint:***
> - The code observations are in column `C_obs_m` and the Doppler observations in column `D_obs_hz`

In [ ]:
# code_min = ...
# code_max = ...
# doppler_min = ...
# doppler_max = ...

# print(f"The GPS L1 C/A code pseudorange observations are within {code_min} and {code_max} meters.")
# print(f"The GPS L1 C/A doppler offset observations are within {doppler_min} and {doppler_max} Hz.")

# Creating sub-groups in a `DataFrame`

The method `.groupby(condition)` creates a `pandas.api.typing.DataFrameGroupBy` object, which is basically the initial dataset split in different groups based on the `condition` criteria. You can then perform operations on the different groups.

The resulting object contains a dictionary `df.groupby(condition).groups` where:
- the keys are the different labels used to categorize the initial dataframe,
- the values are the index corresponding to each label.

> ***Instructions:***
> - Look at the outputs of each of those lines

In [ ]:
data_gps_c1c.groupby("prn").groups
# data_gps_c1c.groupby("prn").groups.keys()
# data_gps_c1c.groupby("prn").groups.values()
# data_gps_c1c.groupby("prn").get_group(2).head()


> ***Instructions:***  
> - plot the number of observations vs time  

> ***Hint:***
> - you can use `.groupby("time_of_reception_in_receiver_time")`
> - the size of all groups can be called using the `.size()` method
> - you can plot the data of a `pd.Series` using the `.plot()` method
> - try to do this plot with single line!!

In [ ]:
# plot the number of observations at each epoch

### Iterating on subgroups created by `.groupby()`

You can iterate on a `DataFrameGroupBy` object by using a `for` loop with the following call:  
> `for key, group in df.groupby("column"):`

This will iterate over all groups created by `.groupby("column")`, with  
- `key` being the value of `"column"` of the current group
- `group` being a `DataFrame` of the current group

> ***Instructions:***  
> Use a `for` loop on `.groupby()`
> - to compute the number of observations at each epoch
> - `print()` the result in the console
>
> ***Hint:***
> - you can use `len(df)` to know the number of rows of a `pd.DataFrame`

In [ ]:
# index=0
# for epoch, group  in TO_BE_REPLACED:
#     print(f"There are {TO_BE_REPLACED} observations at epoch {epoch}")
#     # stop after 5 iterations
#     index+=1
#     if index >= 5:
#         break

# Plot
`pandas` has integrated plotting capabilities. You can easily plot a `Series` by using the `.plot()` method.

> ***Instruction:***  
> - Plot the code observations.

In [ ]:
# Plot the code observations

We can specify the columns to be used as x- or y- axis: `pd.DataFrame.plot(x="axis_1", y="axis_2")`

> ***Instruction:***  
> - Plot the code observation vs time.

In [ ]:
# Plot the code observations vs time

⚠️ We see that since the data from several satellites are all mixed up, we do not distinguish a clear pattern!

To plot add several lines to the same plot, we have to specify that the plots are drawn on the same `matplotlib.pyplot.axes`. This can be done by
- creating an `axes` object with `fig, axis = plt.subplots()`
- specified using the `ax=axis` keyword argument when calling `DataFrame.plot()`

> ***Instruction:***
> - Plot the GPS L1 C/A code observations vs time, with a different color for prn 2, 3 and 8.

In [ ]:
# Plot the code observations of prn 2,3,8 vs time

Do you remember the `.groupby()` method? Try to use it in order to plot the code observations of all prn vs time

> ***Instruction:***
> - Plot the GPS L1 C/A code observations vs time for all prn, using `.groupby()`

> ***Hint:***
> - You could use the for loop call: `for key,group in df.groupby("col"):`, and use the `.plot()` on the resulting group.
> - ... Or you can try to do this in a single line, using `.plot()` directly on the `df.groupby("col")` result!

In [ ]:
# plotting using groupby

# Applying functions to whole `DataFrame`

When dealing with tabular data, it is quite usual to apply a function to each row of the table. This can be done using the `.apply()` ([doc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)) method of a `DataFrame`.

Let's use it to compute the orbital radius of the satellite for each row of the `DataFrame`.
- The orbital radius can be obtained by taking the norm of the satellite ECEF position. The norm of an `numpy.array`  is computed using the function `numpy.linalg.norm(array)`
- The `norm` function has to be applied only to an array containing the values in the columns `["sat_pos_x_m","sat_pos_y_m","sat_pos_z_m"]`
- The `apply` method has to be applied to each row, so we have to define the keyword argument `axis=1` in the call to the `apply()` function.

So in the end, we'll have to use: `data_gps_c1c.satellite_position_m.apply(np.linalg.norm, axis=1)`

> ***Instruction:***
> - Compute the orbital radius of each GPS satellite
>   - select the useful columns
>   - use `.apply(np.linalg.norm, axis=1)` to apply the `norm` function to each row
> - add it as an additional column named `orbital_radius_m` to `data_gps_c1c`
> - plot the orbital radius vs time for each GPS satellite

In [ ]:
# Use .apply() to compute the norm of the satellite position for each row

In [ ]:
# add this data as an additional column to data_gps_c1c
# data_gps_c1c["orbital_radius_m"] = ...

# show the first 3 lines
data_gps_c1c.head(3)

In [ ]:
# plot the orbital radius vs time for each satellite using data_gps_c1c.groupby()

> ***Instructions:***
> - Compute the orbital radius over time for constellations `["G","E","C"]` (using `.apply()` on `data_prx`) and add it as an additional column named `"orbital_radius_m"`
> - Plot the ***mean*** orbital radius as a function of the satellite index (from 0 to $N_{sat}$).
>   - You'll need to use `data_prx.groupby("constellation","prn")`
>   - You can have a look at `data_prx.groupby("constellation","prn").groups.keys()` and `data_prx.groupby("constellation","prn").values()` to understand the result of `.groupby()`
>   - You can use `.mean()` on a `pd.Series` to compute its mean.
> - Can you explain the differences between the satellites/constellations ?

In [ ]:
# add the orbital radius as an additional column to data_prx
# data_prx["orbital_radius_m"] = ...

# plot the the mean orbital radius for each satellite in constellations ["G","E","C"]. The x value is an integer from 0 to Nsat-1

Congratulations, you've reached the end of this notebook!! 🎉

Now, go back to the first cell and load the long data file `"[...]_01D_30S_MO.csv"` and run all cells (There is a button or a Menu action for that).  
It contains a full day of observations, so the plots of the *number of satellites* or *code observations* should be more interesting.